# Carga de datos


In [ ]:
!pip install "vegafusion[embed]>=1.6.0" vegafusion
!pip install "vl-convert-python>=1.6.0"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 49.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 48.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.0/33.0 MB 14.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import altair as alt
import numpy as np

alt.data_transformers.enable("vegafusion")

DataTransformerRegistry.enable('vegafusion')

In [ ]:
!gdown 1RNA2JPEJDzDvruqHwdUzTTBrQRBJOrfJ
df = pd.read_csv('liga_argentina_features_v2.csv', sep=';')

Downloading...
From: https://drive.google.com/uc?id=1RNA2JPEJDzDvruqHwdUzTTBrQRBJOrfJ
To: /content/liga_argentina_features_v2.csv
100% 1.70M/1.70M [00:00<00:00, 23.9MB/s]


# Visualizacion 1-Evolución de la ventaja de la localía

In [ ]:
stream_data = df.groupby(['season', 'Resultado']).size().reset_index(name='cantidad')
stream_data['porcentaje'] = stream_data.groupby('season')['cantidad'].transform(lambda x: x / x.sum() * 100)

viz1 = alt.Chart(stream_data).mark_area(interpolate='monotone').encode(
    x=alt.X('season:O', title='Temporada', axis=alt.Axis(labelAngle=-45, labelFontSize=12)),
    y=alt.Y('porcentaje:Q', title='Porcentaje de Partidos (%)', stack='center', axis=alt.Axis(labelFontSize=12)),
    color=alt.Color('Resultado:N',
                    scale=alt.Scale(domain=['Ganador local', 'Empate', 'Ganador visitante'],
                                   range=['#2ecc71', '#f39c12', '#e74c3c']),
                    legend=alt.Legend(title='Resultado', titleFontSize=14, labelFontSize=12)),
    tooltip=[alt.Tooltip('season:O', title='Temporada'),
             alt.Tooltip('Resultado:N', title='Resultado'),
             alt.Tooltip('cantidad:Q', title='Partidos'),
             alt.Tooltip('porcentaje:Q', title='Porcentaje (%)', format='.1f')]
).properties(
    width=750, height=400,
    title=alt.TitleParams(text='Evolución de la Ventaja de Localía (2015-2025)',
                         fontSize=18, fontWeight='bold', anchor='middle')
).configure_view(strokeWidth=0).configure_axis(grid=True, gridOpacity=0.3)


display(viz1)

alt.Chart(...)

# Visualizacion 2-Comparación de métricas entre dos equipos

In [ ]:
# Datos como local
local_metrics = df.groupby('Equipo_local').agg({
    'local_team_value': 'mean',
    'Victorias_local_en_casa_tasa_normalizada': 'mean',
    'Promedio_Goles_marcados_local_en_casa_normalizado': 'mean',
    'Promedio_Goles_recibidos_local_en_casa_normalizado': 'mean',
    'Valla_invicta_local_tasa_normalizada': 'mean',
    'Promedio_Puntuacion_total_local_normalizado': 'mean'
}).reset_index()

local_metrics.columns = [
    'Equipo', 'Valor', 'Victorias', 'Goles_Marcados',
    'Goles_Recibidos', 'Valla_Invicta', 'Puntuacion'
]

# Datos como visitante
visit_metrics = df.groupby('Equipo_visitante').agg({
    'visitante_team_value': 'mean',
    'Victorias_visitante_fuera_tasa_normalizada': 'mean',
    'Promedio_Goles_marcados_visitante_fuera_normalizado': 'mean',
    'Promedio_Goles_recibidos_visitante_fuera_normalizado': 'mean',
    'Valla_invicta_visitante_tasa_normalizada': 'mean',
    'Promedio_Puntuacion_total_visitante_normalizado': 'mean'
}).reset_index()

visit_metrics.columns = [
    'Equipo', 'Valor', 'Victorias', 'Goles_Marcados',
    'Goles_Recibidos', 'Valla_Invicta', 'Puntuacion'
]

# Combinar y promediar ambas condiciones
all_metrics = pd.concat([local_metrics, visit_metrics])
radar_data = all_metrics.groupby('Equipo').agg({
    'Valor': 'mean',
    'Victorias': 'mean',
    'Goles_Marcados': 'mean',
    'Goles_Recibidos': 'mean',
    'Valla_Invicta': 'mean',
    'Puntuacion': 'mean'
}).reset_index()

# Normalizar el valor del equipo a escala 0-1
radar_data['Valor_Norm'] = (radar_data['Valor'] - radar_data['Valor'].min()) / (
    radar_data['Valor'].max() - radar_data['Valor'].min()
)

radar_data = radar_data.dropna()

# Lista de equipos
equipos_disponibles = sorted(radar_data['Equipo'].unique().tolist())

# Preparar métricas (ahora todas normalizadas entre 0 y 1)
metrics = ['Victorias', 'Goles_Marcados', 'Goles_Recibidos', 'Valla_Invicta', 'Puntuacion', 'Valor_Norm']
metric_labels = ['Victorias', 'Goles A Favor', 'Goles Recibidos', 'Vallas Invictas', 'Puntuación', 'Valor Equipo']

# Crear datos en formato largo
radar_long = []
for _, row in radar_data.iterrows():
    for metric, label in zip(metrics, metric_labels):
        radar_long.append({
            'Equipo': row['Equipo'],
            'Metrica': label,
            'Valor': row[metric]
        })

radar_df = pd.DataFrame(radar_long)

# Calcular coordenadas polares
metric_order = {label: i for i, label in enumerate(metric_labels)}
radar_df['orden'] = radar_df['Metrica'].map(metric_order)
radar_df['angulo'] = (radar_df['orden'] / len(metric_labels)) * 2 * np.pi
radar_df['x'] = radar_df['Valor'] * np.cos(radar_df['angulo'])
radar_df['y'] = radar_df['Valor'] * np.sin(radar_df['angulo'])

# Cerrar el polígono para cada equipo
radar_df_closed = []
for equipo in radar_df['Equipo'].unique():
    equipo_data = radar_df[radar_df['Equipo'] == equipo].copy()
    first_point = equipo_data.iloc[0:1].copy()
    radar_df_closed.append(pd.concat([equipo_data, first_point]))

radar_df_closed = pd.concat(radar_df_closed, ignore_index=True)

# Crear datos auxiliares
axis_data = []
for label in metric_labels:
    orden = metric_order[label]
    angulo = (orden / len(metric_labels)) * 2 * np.pi
    axis_data.append({
        'Metrica': label,
        'x': 0, 'y': 0,
        'x2': np.cos(angulo),
        'y2': np.sin(angulo)
    })
axis_df = pd.DataFrame(axis_data)

label_data = []
for label in metric_labels:
    orden = metric_order[label]
    angulo = (orden / len(metric_labels)) * 2 * np.pi
    label_data.append({
        'Metrica': label,
        'x': 1.2 * np.cos(angulo),
        'y': 1.2 * np.sin(angulo)
    })
label_df = pd.DataFrame(label_data)

circle_data = []
for r in [0.2, 0.4, 0.6, 0.8, 1.0]:
    for i in range(len(metric_labels) + 1):
        angulo = (i / len(metric_labels)) * 2 * np.pi
        circle_data.append({
            'radio': r,
            'x': r * np.cos(angulo),
            'y': r * np.sin(angulo)
        })
circle_df = pd.DataFrame(circle_data)

# Crear selecciones de equipos
equipo1_select = alt.selection_point(
    name='equipo1',
    fields=['Equipo'],
    bind=alt.binding_select(
        options=[None] + equipos_disponibles,
        labels=['Seleccionar...'] + equipos_disponibles,
        name='Equipo 1: '
    ),
    value=[{'Equipo': equipos_disponibles[0]}]
)

equipo2_select = alt.selection_point(
    name='equipo2',
    fields=['Equipo'],
    bind=alt.binding_select(
        options=[None] + equipos_disponibles,
        labels=['Seleccionar...'] + equipos_disponibles,
        name='Equipo 2: '
    ),
    value=[{'Equipo': equipos_disponibles[1] if len(equipos_disponibles) > 1 else equipos_disponibles[0]}]
)

# Círculos de referencia
circles = alt.Chart(circle_df).mark_line(
    strokeWidth=0.5, stroke='lightgray', opacity=0.5
).encode(
    x=alt.X('x:Q', scale=alt.Scale(domain=[-1.4, 1.4]), axis=None),
    y=alt.Y('y:Q', scale=alt.Scale(domain=[-1.4, 1.4]), axis=None),
    detail='radio:Q'
)

# Ejes
axes = alt.Chart(axis_df).mark_rule(
    strokeWidth=1, stroke='gray', opacity=0.5
).encode(x='x:Q', y='y:Q', x2='x2:Q', y2='y2:Q')

# Labels
labels = alt.Chart(label_df).mark_text(
    fontSize=12, fontWeight='bold'
).encode(x='x:Q', y='y:Q', text='Metrica:N')

# Polígono Equipo 1
radar1 = alt.Chart(radar_df_closed).mark_line(
    point=True, strokeWidth=3, filled=True, opacity=0.3
).encode(
    x='x:Q', y='y:Q',
    order='orden:Q',
    color=alt.value('#1f77b4'), # Color for team 1
    tooltip=[
        alt.Tooltip('Equipo:N'),
        alt.Tooltip('Metrica:N'),
        alt.Tooltip('Valor:Q', format='.3f', title='Valor (0-1)')
    ]
).add_params(
    equipo1_select
).transform_filter(
    equipo1_select
)

# Polígono Equipo 2
radar2 = alt.Chart(radar_df_closed).mark_line(
    point=True, strokeWidth=3, filled=True, opacity=0.3
).encode(
    x='x:Q', y='y:Q',
    order='orden:Q',
    color=alt.value('#ff7f0e'), # Color for team 2
    tooltip=[
        alt.Tooltip('Equipo:N'),
        alt.Tooltip('Metrica:N'),
        alt.Tooltip('Valor:Q', format='.3f', title='Valor (0-1)')
    ]
).add_params(
    equipo2_select
).transform_filter(
    equipo2_select
)

# Create dynamic legend using text and square marks
legend_data = pd.DataFrame({
    'label': ['Equipo 1', 'Equipo 2'],
    'y_pos': [0, 1],
    'color': ['#1f77b4', '#ff7f0e']
})

legend_squares = alt.Chart(legend_data).mark_square(size=100).encode(
    y=alt.Y('y_pos:O', axis=None),
    color=alt.Color('color:N', scale=None),
    tooltip=alt.Tooltip('label:N')
)

legend_text = alt.Chart(legend_data).mark_text(align='left', baseline='middle', dx=15).encode(
    y=alt.Y('y_pos:O', axis=None),
    text=alt.Text('label:N'),
    color=alt.value('black') # Set text color to black
).transform_calculate(
    label="datum.label == 'Equipo 1' ? equipo1.Equipo : equipo2.Equipo" # Use calculated field for text based on selection
)


# Combine legend squares and text
legend = (legend_squares + legend_text).properties(title='Equipos')


# Combinar todo
radar_chart = (circles + axes + labels + radar1 + radar2).properties(
    width=700,
    height=700,
    title={
        'text': 'Comparación: Selecciona dos equipos',
    }
)

# Combine the radar chart with the legend
final_chart = alt.hconcat(radar_chart, legend).configure_view(strokeWidth=0)

final_chart

alt.HConcatChart(...)

# Visualizacion 3-Comparación de ultimos 5 partidos entre dos equipos para una fecha especifica

In [ ]:
import numpy as np

# Preparar datos de forma reciente
forma_data = df[['fecha_del_partido', 'Equipo_local', 'Equipo_visitante',
                 'Forma_local_puntos_ultimos5', 'Forma_visitante_puntos_ultimos5',
                 'Forma_local_ultimos5', 'Forma_visitante_ultimos5']].copy()

forma_data['fecha'] = pd.to_datetime(forma_data['fecha_del_partido'])

# Crear dataset combinado
forma_local = forma_data[['fecha', 'Equipo_local', 'Forma_local_puntos_ultimos5', 'Forma_local_ultimos5']].copy()
forma_local.columns = ['fecha', 'Equipo', 'Puntos_Ultimos5', 'Forma_Ultimos5']

forma_visitante = forma_data[['fecha', 'Equipo_visitante', 'Forma_visitante_puntos_ultimos5', 'Forma_visitante_ultimos5']].copy()
forma_visitante.columns = ['fecha', 'Equipo', 'Puntos_Ultimos5', 'Forma_Ultimos5']

forma_combined = pd.concat([forma_local, forma_visitante])
forma_combined = forma_combined.dropna(subset=['Puntos_Ultimos5'])
forma_combined = forma_combined.sort_values('fecha')
forma_combined['fecha_str'] = forma_combined['fecha'].dt.strftime('%Y-%m-%d')

# Listas para selectores
equipos_disponibles = sorted(forma_combined['Equipo'].unique().tolist())
fechas_disponibles = sorted(forma_combined['fecha_str'].unique().tolist())

# Valores por defecto
equipo1_default = equipos_disponibles[0]
equipo2_default = equipos_disponibles[1] if len(equipos_disponibles) > 1 else equipos_disponibles[0]
fecha_default = fechas_disponibles[-1]

# Filtrar datos para los valores por defecto
def get_data_for_comparison(df, equipo1, equipo2, fecha):
    data_eq1 = df[(df['Equipo'] == equipo1) & (df['fecha_str'] == fecha)]
    data_eq2 = df[(df['Equipo'] == equipo2) & (df['fecha_str'] == fecha)]
    return data_eq1, data_eq2

# Crear widget interactivo usando ipywidgets
from ipywidgets import interact, widgets

def plot_comparison(Equipo_1, Equipo_2, Fecha):
    import matplotlib.pyplot as plt
    import matplotlib.patches as mpatches

    # Filtrar datos
    data_eq1 = forma_combined[(forma_combined['Equipo'] == Equipo_1) &
                              (forma_combined['fecha_str'] == Fecha)]
    data_eq2 = forma_combined[(forma_combined['Equipo'] == Equipo_2) &
                              (forma_combined['fecha_str'] == Fecha)]

    if data_eq1.empty:
        print(f"⚠️ No hay datos para {Equipo_1} en la fecha {Fecha}")
        data_eq1 = forma_combined[forma_combined['Equipo'] == Equipo_1].tail(1)
        if not data_eq1.empty:
            print(f"   Mostrando último dato disponible: {data_eq1['fecha_str'].values[0]}")

    if data_eq2.empty:
        print(f"⚠️ No hay datos para {Equipo_2} en la fecha {Fecha}")
        data_eq2 = forma_combined[forma_combined['Equipo'] == Equipo_2].tail(1)
        if not data_eq2.empty:
            print(f"   Mostrando último dato disponible: {data_eq2['fecha_str'].values[0]}")

    if data_eq1.empty or data_eq2.empty:
        print("❌ No se pueden mostrar datos para la comparación")
        return

    # Crear figura
    fig, axes = plt.subplots(3, 1, figsize=(12, 8), gridspec_kw={'height_ratios': [2, 1, 1]})
    fig.suptitle('Comparación de Forma Reciente entre Equipos', fontsize=16, fontweight='bold')

    # GRÁFICO 1: Barras de puntos
    ax1 = axes[0]

    puntos_eq1 = data_eq1['Puntos_Ultimos5'].values[0]
    puntos_eq2 = data_eq2['Puntos_Ultimos5'].values[0]

    bars = ax1.barh([Equipo_1, Equipo_2], [puntos_eq1, puntos_eq2],
                    color=['#4472C4', '#ED7D31'], height=0.5)

    ax1.set_xlim(0, 15)
    ax1.set_xlabel('Puntos (últimos 5 partidos)', fontsize=12)
    ax1.set_title('Puntos Obtenidos', fontsize=13, fontweight='bold')
    ax1.grid(axis='x', alpha=0.3)

    # Añadir valores en las barras
    for i, (bar, puntos) in enumerate(zip(bars, [puntos_eq1, puntos_eq2])):
        ax1.text(puntos + 0.3, bar.get_y() + bar.get_height()/2,
                f'{int(puntos)} pts', va='center', fontweight='bold', fontsize=11)

    # Líneas de referencia
    ax1.axvline(x=15, color='green', linestyle='--', alpha=0.4, linewidth=1)
    ax1.axvline(x=10, color='blue', linestyle='--', alpha=0.3, linewidth=1)
    ax1.axvline(x=7.5, color='gray', linestyle='--', alpha=0.3, linewidth=1)

    # GRÁFICO 2: Racha Equipo 1
    ax2 = axes[1]
    forma_eq1 = str(data_eq1['Forma_Ultimos5'].values[0]) if pd.notna(data_eq1['Forma_Ultimos5'].values[0]) else 'N/A'

    ax2.set_xlim(0, 5)
    ax2.set_ylim(0, 1)
    ax2.set_title(f'Racha Equipo 1: {Equipo_1}', fontsize=12, fontweight='bold')
    ax2.axis('off')

    if forma_eq1 != 'N/A':
        for i, resultado in enumerate(forma_eq1):
            color = {'W': '#2ca02c', 'D': '#ffcc00', 'L': '#d62728'}.get(resultado, 'gray')
            rect = mpatches.Rectangle((i, 0.2), 0.8, 0.6, linewidth=2,
                                     edgecolor='white', facecolor=color)
            ax2.add_patch(rect)
            texto = {'W': 'V', 'D': 'E', 'L': 'D'}.get(resultado, '?')
            ax2.text(i + 0.4, 0.5, texto, ha='center', va='center',
                    fontsize=14, fontweight='bold', color='white')

    ax2.text(2.5, -0.2, '← Más antiguo    Más reciente →', ha='center',
            fontsize=9, style='italic', color='gray')

    # GRÁFICO 3: Racha Equipo 2
    ax3 = axes[2]
    forma_eq2 = str(data_eq2['Forma_Ultimos5'].values[0]) if pd.notna(data_eq2['Forma_Ultimos5'].values[0]) else 'N/A'

    ax3.set_xlim(0, 5)
    ax3.set_ylim(0, 1)
    ax3.set_title(f'Racha Equipo 2: {Equipo_2}', fontsize=12, fontweight='bold')
    ax3.axis('off')

    if forma_eq2 != 'N/A':
        for i, resultado in enumerate(forma_eq2):
            color = {'W': '#2ca02c', 'D': '#ffcc00', 'L': '#d62728'}.get(resultado, 'gray')
            rect = mpatches.Rectangle((i, 0.2), 0.8, 0.6, linewidth=2,
                                     edgecolor='white', facecolor=color)
            ax3.add_patch(rect)
            texto = {'W': 'V', 'D': 'E', 'L': 'D'}.get(resultado, '?')
            ax3.text(i + 0.4, 0.5, texto, ha='center', va='center',
                    fontsize=14, fontweight='bold', color='white')

    ax3.text(2.5, -0.2, '← Más antiguo    Más reciente →', ha='center',
            fontsize=9, style='italic', color='gray')

    # Leyenda
    victoria_patch = mpatches.Patch(color='#2ca02c', label='Victoria (V)')
    empate_patch = mpatches.Patch(color='#ffcc00', label='Empate (E)')
    derrota_patch = mpatches.Patch(color='#d62728', label='Derrota (D)')
    fig.legend(handles=[victoria_patch, empate_patch, derrota_patch],
              loc='lower center', ncol=3, bbox_to_anchor=(0.5, -0.02))

    plt.tight_layout()
    plt.show()

    # Mostrar estadísticas adicionales
    print(f"\n📊 Estadísticas Detalladas:")
    print(f"{'='*60}")
    print(f"🔵 {Equipo_1}: {int(puntos_eq1)} puntos | Racha: {forma_eq1}")
    print(f"🟠 {Equipo_2}: {int(puntos_eq2)} puntos | Racha: {forma_eq2}")
    print(f"📅 Fecha: {Fecha}")

    diferencia = abs(puntos_eq1 - puntos_eq2)
    mejor = Equipo_1 if puntos_eq1 > puntos_eq2 else Equipo_2
    if puntos_eq1 == puntos_eq2:
        print(f"⚖️  Ambos equipos tienen la misma forma reciente")
    else:
        print(f"🏆 {mejor} tiene mejor forma reciente (+{int(diferencia)} puntos)")

# Crear selectores interactivos
interact(plot_comparison,
         Equipo_1=widgets.Dropdown(options=equipos_disponibles, value=equipo1_default, description='🔵 Equipo 1:'),
         Equipo_2=widgets.Dropdown(options=equipos_disponibles, value=equipo2_default, description='🟠 Equipo 2:'),
         Fecha=widgets.Dropdown(options=fechas_disponibles[-100:], value=fecha_default, description='📅 Fecha:'))

interactive(children=(Dropdown(description='🔵 Equipo 1:', options=('Aldosivi', 'Argentinos JRS', 'Arsenal Sara…

<function __main__.plot_comparison(Equipo_1, Equipo_2, Fecha)>

# Visulización 4-Relación del Valor de Mercado vs Rendimiento

In [ ]:
equipos_stats = []
for season in df['season'].unique():
    season_df = df[df['season'] == season]
    for equipo in season_df['Equipo_local'].unique():
        local_games = season_df[season_df['Equipo_local'] == equipo]
        away_games = season_df[season_df['Equipo_visitante'] == equipo]
        wins_local = (local_games['Resultado'] == 'Ganador local').sum()
        wins_away = (away_games['Resultado'] == 'Ganador visitante').sum()
        total_games = len(local_games) + len(away_games)
        total_wins = wins_local + wins_away
        valor = local_games['local_team_value'].iloc[0] if len(local_games) > 0 else (
            away_games['visitante_team_value'].iloc[0] if len(away_games) > 0 else None)
        if total_games > 0 and pd.notna(valor):
            equipos_stats.append({
                'equipo': equipo, 'season': season, 'victorias': total_wins, 'partidos': total_games,
                'porcentaje_victorias': (total_wins / total_games) * 100, 'valor_mercado': valor
            })

equipos_df = pd.DataFrame(equipos_stats)
cinco_grandes = ['River Plate', 'Boca Juniors', 'Racing Club', 'San Lorenzo', 'Independiente']
equipos_df['categoria'] = equipos_df['equipo'].apply(lambda x: '5 Grandes' if x in cinco_grandes else 'Otros')

# Selector de temporada con dropdown
season_dropdown = alt.binding_select(
    options=sorted(equipos_df['season'].unique().tolist()),
    name='Temporada: '
)
season_select = alt.selection_point(fields=['season'], bind=season_dropdown, value=2025)

# Scatter plot que solo muestra la temporada seleccionada
scatter = alt.Chart(equipos_df).mark_circle(size=150, opacity=0.85).encode(
    x=alt.X('valor_mercado:Q', title='Valor de Mercado (millones €)', scale=alt.Scale(zero=False),
            axis=alt.Axis(labelFontSize=12, titleFontSize=13)),
    y=alt.Y('porcentaje_victorias:Q', title='Porcentaje de Victorias (%)',
            axis=alt.Axis(labelFontSize=12, titleFontSize=13)),
    color=alt.Color('categoria:N',
                   scale=alt.Scale(domain=['5 Grandes', 'Otros'], range=['#e74c3c', '#3498db']),
                   legend=alt.Legend(title='Categoría', titleFontSize=14, labelFontSize=12)),
    size=alt.Size('partidos:Q', scale=alt.Scale(range=[100, 500]),
                 legend=alt.Legend(title='Partidos', titleFontSize=12)),
    tooltip=[
        alt.Tooltip('equipo:N', title='Equipo'),
        alt.Tooltip('season:O', title='Temporada'),
        alt.Tooltip('valor_mercado:Q', title='Valor (M€)', format='.1f'),
        alt.Tooltip('porcentaje_victorias:Q', title='% Victorias', format='.1f'),
        alt.Tooltip('victorias:Q', title='Victorias'),
        alt.Tooltip('partidos:Q', title='Partidos')
    ]
).transform_filter(
    season_select
).add_params(season_select)

# Línea de regresión para la temporada seleccionada
regression = scatter.transform_regression(
    'valor_mercado', 'porcentaje_victorias', method='linear'
).mark_line(color='#e67e22', strokeWidth=3, strokeDash=[5, 5])

viz2 = (scatter + regression).properties(
    width=750, height=500,
    title=alt.TitleParams(
        text='Relación entre Valor de Mercado y Rendimiento',
        subtitle='Selecciona una temporada del menú | Tamaño del círculo = cantidad de partidos',
        fontSize=18, fontWeight='bold', anchor='middle'
    )
).configure_view(strokeWidth=0)

display(viz2)

alt.LayerChart(...)

# Visualización 5-Forma Reciente vs Diferencia de Goles

In [ ]:
forma_data = df[
    (df['Forma_local_puntos_ultimos5'].notna()) &
    (df['Forma_visitante_puntos_ultimos5'].notna()) &
    (df['Promedio_Diferencia_gol_total_local_normalizado'].notna()) &
    (df['Promedio_Diferencia_gol_total_visitante_normalizado'].notna())
].copy()

forma_data['diferencia_forma'] = forma_data['Forma_local_puntos_ultimos5'] - forma_data['Forma_visitante_puntos_ultimos5']
forma_data['diferencia_goles_norm'] = forma_data['Promedio_Diferencia_gol_total_local_normalizado'] - forma_data['Promedio_Diferencia_gol_total_visitante_normalizado']

# Categorizar la diferencia de forma en rangos para mejor visualización
def categorizar_forma(diff):
    if diff <= -6:
        return 'Visitante muy superior'
    elif diff <= -3:
        return 'Visitante superior'
    elif diff < 3:
        return 'Equilibrado'
    elif diff < 6:
        return 'Local superior'
    else:
        return 'Local muy superior'

forma_data['categoria_forma'] = forma_data['diferencia_forma'].apply(categorizar_forma)

# Calcular estadísticas por categoría y resultado
stats_forma = forma_data.groupby(['categoria_forma', 'Resultado']).size().reset_index(name='cantidad')
stats_forma_total = forma_data.groupby('categoria_forma').size().reset_index(name='total')
stats_forma = stats_forma.merge(stats_forma_total, on='categoria_forma')
stats_forma['porcentaje'] = (stats_forma['cantidad'] / stats_forma['total']) * 100

# Ordenar categorías
orden_categorias = ['Visitante muy superior', 'Visitante superior', 'Equilibrado', 'Local superior', 'Local muy superior']
stats_forma['categoria_forma'] = pd.Categorical(stats_forma['categoria_forma'], categories=orden_categorias, ordered=True)
stats_forma = stats_forma.sort_values('categoria_forma')

# Gráfico de barras apiladas normalizado
barras = alt.Chart(stats_forma).mark_bar().encode(
    x=alt.X('categoria_forma:N',
            title='Diferencia de Forma Reciente (últimos 5 partidos)',
            axis=alt.Axis(labelAngle=-20, labelFontSize=10, titleFontSize=12),
            sort=orden_categorias),
    y=alt.Y('porcentaje:Q',
            title='Distribución de Resultados (%)',
            stack='normalize',
            axis=alt.Axis(labelFontSize=11, titleFontSize=12, format='%')),
    color=alt.Color('Resultado:N',
                   scale=alt.Scale(domain=['Ganador local', 'Empate', 'Ganador visitante'],
                                  range=['#2ecc71', '#f39c12', '#e74c3c']),
                   legend=alt.Legend(title='Resultado', titleFontSize=13, labelFontSize=11, orient='top')),
    tooltip=[
        alt.Tooltip('categoria_forma:N', title='Categoría Forma'),
        alt.Tooltip('Resultado:N', title='Resultado'),
        alt.Tooltip('cantidad:Q', title='Partidos'),
        alt.Tooltip('total:Q', title='Total en categoría'),
        alt.Tooltip('porcentaje:Q', title='Porcentaje', format='.1f')
    ]
)

# Texto con los totales
text = alt.Chart(stats_forma_total).mark_text(dy=-10, fontSize=11, fontWeight='bold').encode(
    x=alt.X('categoria_forma:N', sort=orden_categorias),
    y=alt.value(0),
    text=alt.Text('total:Q', format='d')
)

viz3a = (barras + text).properties(
    width=750, height=400,
    title=alt.TitleParams(
        text='¿El Momentum Reciente Predice el Resultado?',
        subtitle=f'Total de {len(forma_data)} partidos analizados | Números arriba = cantidad de partidos en cada categoría',
        fontSize=18, fontWeight='bold', anchor='middle'
    )
).configure_view(strokeWidth=0)

display(viz3a)

alt.LayerChart(...)